## Paquets i dades originals ## 

In [2]:
import json
import pandas as pd
from datetime import datetime


from auxiliary_functions import (obtenir_data_presencia_codi, restar_dates, codis_ICD, nombre_ingressos,
                                 dies_ingressat_total, interval_10_edat,
                                 sumar_barthel, sumar_emina, obtenir_ultima_clau, obtenir_pes_o_mitjana,
                                 canadenca_comparada, disfagia_mecvvs, extreure_valors_claus,
                                 extreure_valors_claus_simple, obtenir_valors_clau_interes, index_charlson,
                                 obtenir_pes_mes_antic, obtenir_pes_mes_nou, obtenir_data_pes_mes_antic,
                                 obtenir_primera_data_mecvv, obtenir_pes_coincident_mecvv, restar_columnes_object)
from listas import (PA_list, P_list, disfagia_list, Main_respiratory_infections_list, LRTI_list,
                    COPD_exacerbations_list,
                    Pulmonary_fibrosis_fibrotorax_list, priorfalls_list, delirium_list, dementia_list, depresyndr_list,
                    uriincont_list, fecincont_list, pressulc_list, immob_list, conf_list, osteopor_list,
                    sarcopenia_list,
                    sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list,
                    ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list,
                    psicosis_list, nutridef_list, charlson_dict)

taula = pd.read_pickle('./data/processed/dataframe.pkl')
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


Prèviament, he tractat les dades creant noves columnes que classifiquen millor tota la informació disponible

In [3]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


Distribuiré les dades en 3 grups (més un grup que conté la totalitat de dades):
- **AMB_PA**: aquest grup està compost per pacients que contenen el codi ICD de pneumònia aspirativa al seu historial.
- **AMB_PA_MECVV**: aquest grup conté els pacients que compleixen les següents característiques: 
    - Han donat positiu al MECVV (és a dir, que tenen disfàgia i una alteració en la seguretat o en l'eficàcia)
    - Han sigut diagnosticats amb el codi ICD de pneumònia
    - Han passat menys de 30 dies entre el diagnòstic amb l'ICD de pneumònia i el resultat positiu al MECVV
- **SENSE_PA**: aquest grup està format per pacients que contenen el codi ICD de pneumònia o bé:
    - Contenen el codi ICD de pneumònia
    - Han donat positiu al MECVV 
    - Han passat més de 30 dies entre el diagnòstic amb l'ICD de pneumònia i  el resultat positiu del MECVV 
- **TOTAL**: aquest grup conté tots els pacients de l'estudi *#TODO: hacer descriptiva de este grupo*

# AMB_PA 

In [4]:
AMB_PA = taula[taula["PA diagnosticada"] == 1.0]
AMB_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,2021-08-19,None,None,None,2021-08-17,2.0
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,46.4,46.4,2020-09-09,2020-09-07,46.4,0.0,0.0,2020-09-04,3.0


# AMB_PA_MECVV

In [5]:
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)] 
AMB_PA_MECVV.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,None,None,None,None,2021-08-19,None,None,None,2021-08-17,2.0
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,None,46.4,46.4,2020-09-09,2020-09-07,46.4,0.0,0.0,2020-09-04,3.0
27,100,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2021-09-14'}, {'valor...","[{'codi': 'C08CA', 'dataInici': '2020-11-23 00...","[{'data': '20201125131827', 'disfagia': 'S', '...",...,None,41.100,0,2021-01-20,2020-11-25,None,None,41.1,2020-12-02,7.0
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,48,0,0,2022-05-30,2022-05-29,0,0.0,0.0,2022-05-23,6.0


# SENSE_PA

In [6]:
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) & 
           (taula['P diagnosticada'] == 1.0)]
SENSE_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,70,None,None,None,2020-09-07,None,None,None,2022-02-14,525.0
25,83,F,0,"[{'date': '20220308', 'name': 'UREA Sèrum', 'l...","[{'imc': '', 'anorexia': '', 'perduaPes': '', ...","[{'activitat': '0', 'estatMental': '1', 'humit...",[],"[{'valor': '57', 'data': '2022-11-03'}, {'valo...","[{'codi': 'C10AA', 'dataInici': '2022-09-27 00...","[{'data': '20220929091303', 'disfagia': 'S', '...",...,38,57,63.2,2022-11-03,2022-09-29,None,None,-6.2,2023-04-09,192.0
31,89,M,0,"[{'date': '20220318', 'name': 'BILIRUBINA TOTA...","[{'imc': '', 'anorexia': '', 'perduaPes': '', ...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '75', 'data': '2022-01-11'}, {'valo...","[{'codi': 'C10AA', 'dataInici': '2021-12-23 00...","[{'data': '20220119101523', 'disfagia': 'N', '...",...,78,75,67,2022-01-11,2022-02-01,None,None,8.0,2023-05-05,458.0
39,87,F,0,"[{'date': '20230619', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2021-01-17 00...","[{'data': '20220111085147', 'disfagia': 'S', '...",...,>90,None,None,None,2022-01-11,None,None,None,2023-06-19,524.0
41,93,F,0,"[{'date': '20221013', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '0', 'data': '2023-06-02'}, {'valor...","[{'codi': 'C10AA', 'dataInici': '2021-02-05 00...","[{'data': '20230530155654', 'disfagia': 'S', '...",...,48,0,60,2023-06-02,2023-05-30,0,0.0,-60.0,2023-03-21,70.0


# TOTAL

In [7]:
TOTAL = taula
taula.head() 

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


## Nombre de pacients

In [8]:
num_AMB_PA = len(AMB_PA)
num_AMB_PA_MECVV= len(AMB_PA_MECVV)
num_SENSE_PA = len(SENSE_PA)
num_TOTAL = len(TOTAL)

print(num_AMB_PA, num_AMB_PA_MECVV, num_SENSE_PA, num_TOTAL)

17 8 8 50


#### Comparació distributiva

##### Normalitat i p-valor

Ara que tinc totes les dades classificades en aquests grups, faré una descriptiva. També compararé els diferents grups amb el pvalor mitjançant T-test, test de Mann-Whitney o test Xi-quadrat per mirar la seva distribució, la qual esperaré que sigui similar entre el grup AMB_PA i AMB_PA_MECVV, i diferent entre SENSE_PA i els altres 2 grups. Els criteris que seguiré per utilitzar un test o un altre és, primer, fer un test de Shapiro-Wilk per comprovar si segueixen una distribució normal, si és així, usaré el T-test, si no, utilitzaré el de Mann-Whitney. Finalment, si la columna conté valors categòrics com és el cas del sexe (F/M), faré servir el test Xi-quadrat.
Les meves hipòtesis són, pel test de Shapiro-Wilk:
    - H0: les dades segueixen una distribució normal
    - H1: les dades no segueixen una distribució normal
I pels tests (T-test, Mann-Whitney i Xi-quadrat):
    - H0: les dades dels dos grups no són significativament diferents
    - H1: les dades dels dos grups són significativament diferents

## Edat

In [9]:
edat_AMB_PA = AMB_PA['edat'].mean()
desviacio_estandard_edat = AMB_PA['edat'].std()

resultat = f"{edat_AMB_PA:.2f} ± {desviacio_estandard_edat:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultat)


Mitjana ± Desviació Estàndard de l'edat: 87.06 ± 8.99


In [10]:
edat_AMB_PA_MECVV = AMB_PA_MECVV['edat'].mean()
desviacio_estandard_edat = AMB_PA_MECVV['edat'].std()

resultat = f"{edat_AMB_PA_MECVV:.2f} ± {desviacio_estandard_edat:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultat)

Mitjana ± Desviació Estàndard de l'edat: 91.50 ± 6.82


In [11]:
edat_SENSE_PA = SENSE_PA['edat'].mean()
desviacio_estandard_edat = SENSE_PA['edat'].std()

resultat = f"{edat_SENSE_PA:.2f} ± {desviacio_estandard_edat:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultat)

Mitjana ± Desviació Estàndard de l'edat: 86.25 ± 6.90


In [12]:
edat_TOTAL = TOTAL['edat'].mean()
desviacio_estandard_TOTAL = TOTAL['edat'].std()

resultat = f"{edat_SENSE_PA:.2f} ± {desviacio_estandard_edat:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultat)

Mitjana ± Desviació Estàndard de l'edat: 86.25 ± 6.90


#### Pvalor

##### Normalitat 

In [13]:
stat, p_value = shapiro(AMB_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9410
Valor p = 0.3297
El nombre de files segueix una distribució normal.


In [14]:
stat, p_value = shapiro(AMB_PA_MECVV['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8304
Valor p = 0.0600
El nombre de files segueix una distribució normal.


In [15]:
stat, p_value = shapiro(SENSE_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8318
Valor p = 0.0619
El nombre de files segueix una distribució normal.


In [16]:
stat, p_value = shapiro(TOTAL['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8491
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [17]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['edat'], AMB_PA_MECVV['edat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.18896133816771837


In [18]:
stat, p_value_13 = mannwhitneyu(AMB_PA['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.9534289321662349


In [19]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.37026415517950617


In [20]:
stat, p_value_14 = mannwhitneyu(AMB_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.20417514607442822


In [21]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.05636751107260842


In [22]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.4769108663506575


## Sexe

In [23]:
sexe_AMB_PA = AMB_PA['sexe'].value_counts()
sexe_AMB_PA_MECVV = AMB_PA_MECVV['sexe'].value_counts()
sexe_SENSE_PA = SENSE_PA['sexe'].value_counts()
sexe_TOTAL = TOTAL['sexe'].value_counts()

In [24]:
if 'F' in sexe_AMB_PA:
    percent_female1 = (sexe_AMB_PA['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA")

Percentatge de sexe femení en AMB_PA: 47.06%


In [25]:
if 'F' in sexe_AMB_PA_MECVV:
    percent_female2 = (sexe_AMB_PA_MECVV['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA_MECVV: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA_MECVV")

Percentatge de sexe femení en AMB_PA_MECVV: 47.06%


In [26]:
if 'F' in sexe_SENSE_PA:
    percent_female3 = (sexe_SENSE_PA['F'] / len(SENSE_PA)) * 100
    print(f"Percentatge de sexe femení en SENSE_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en SENSE_PA")


Percentatge de sexe femení en SENSE_PA: 47.06%


In [27]:
if 'F' in sexe_TOTAL:
    percent_female3 = (sexe_TOTAL['F'] / len(TOTAL)) * 100
    print(f"Percentatge de sexe femení en TOTAL: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en TOTAL")

Percentatge de sexe femení en TOTAL: 47.06%


#### Pvalor

##### Normalitat

In [28]:
#TODO: ???Al ser una variable categòrica no es pot comprovar la seva normalitat. Realitzaré directament el test Xi-quadrat

In [29]:
contingency_table_AMB_PA_AMB_PA_MECVV = pd.crosstab(index=AMB_PA['sexe'], columns=AMB_PA_MECVV['sexe'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_AMB_PA_AMB_PA_MECVV)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.


In [30]:
contingency_table_AMB_PA_SENSE_PA = pd.crosstab(index=AMB_PA['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.


In [31]:
contingency_table_AMB_PA_MECVV_SENSE_PA = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_MECVV_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [32]:
contingency_table_14 = pd.crosstab(index=AMB_PA['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.


In [33]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.


In [34]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['sexe'], columns=TOTAL['sexe'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.5
P-valor entre SENSE_PA i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.


## Ingressos (admissions)

In [35]:
Mitjana_ingresos = AMB_PA['Nombre ingressos'].mean()
desviacio_estandard_ingresos = AMB_PA['Nombre ingressos'].std()

resultat = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultat)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.59 ± 3.37


In [36]:
Mitjana_ingresos = AMB_PA_MECVV['Nombre ingressos'].mean()
desviacio_estandard_ingresos = AMB_PA_MECVV['Nombre ingressos'].std()

resultat = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultat)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.88 ± 1.64


In [37]:
Mitjana_ingresos = SENSE_PA['Nombre ingressos'].mean()
desviacio_estandard_ingresos = SENSE_PA['Nombre ingressos'].std()

resultat = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultat)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 8.75 ± 3.85


In [38]:
Mitjana_ingresos = TOTAL['Nombre ingressos'].mean()
desviacio_estandard_ingresos = TOTAL['Nombre ingressos'].std()

resultat = f"{Mitjana_ingresos:.2f} ± {desviacio_estandard_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultat)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 6.68 ± 5.08


#### Pvalor

##### Normalitat

In [39]:
stat, p_value = shapiro(AMB_PA['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8008
Valor p = 0.0021
El nombre de files no segueix una distribució normal.


In [40]:
stat, p_value = shapiro(AMB_PA_MECVV['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9122
Valor p = 0.3695
El nombre de files segueix una distribució normal.


In [41]:
stat, p_value = shapiro(SENSE_PA['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9498
Valor p = 0.7091
El nombre de files segueix una distribució normal.


In [42]:
stat, p_value = shapiro(TOTAL['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7256
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [43]:
stat, p_value = shapiro(TOTAL['Nombre ingressos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7256
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [44]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Nombre ingressos'], AMB_PA_MECVV['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.27403132109149697


In [45]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Nombre ingressos'], SENSE_PA['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.042821773602008555


In [46]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Nombre ingressos'], SENSE_PA['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.18434809692120802


In [47]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.4941763107233279


In [48]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.6902658476564807


In [49]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Nombre ingressos'], TOTAL['Nombre ingressos'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.06577367478634001


## Disfàgia

In [50]:
disf_AMB_PA= (AMB_PA['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA:.2f}%")

Percentatge de disfàgia: 47.06%


In [51]:
disf_AMB_PA_MECVV = (AMB_PA_MECVV['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA_MECVV:.2f}%")

Percentatge de disfàgia: 50.00%


In [52]:
disf_SENSE_PA= (SENSE_PA['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_SENSE_PA:.2f}%")

Percentatge de disfàgia: 25.00%


In [53]:
disf_TOTAL= (TOTAL['DO diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_TOTAL:.2f}%")

Percentatge de disfàgia: 20.00%


#### Pvalor

##### Normalitat

In [54]:
stat, p_value = shapiro(AMB_PA['DO diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [55]:
stat, p_value = shapiro(AMB_PA_MECVV['DO diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [56]:
stat, p_value = shapiro(SENSE_PA['DO diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [57]:
stat, p_value = shapiro(TOTAL['DO diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4904
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [58]:
contingency_table_12 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=AMB_PA_MECVV['DO diagnosticada'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.


In [59]:
contingency_table_13 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=SENSE_PA['DO diagnosticada'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.


In [60]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['DO diagnosticada'], columns=SENSE_PA['DO diagnosticada'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [61]:
contingency_table_14 = pd.crosstab(index=AMB_PA['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.


In [62]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.


In [63]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['DO diagnosticada'], columns=TOTAL['DO diagnosticada'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 3.5555555555555554
P-valor entre SENSE_PA i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.


## Index de Barthel

### Mitjana

In [64]:
Mitjana_barthel = AMB_PA['Barthel resultats'].mean()
desviacio_estandard_barthel = AMB_PA['Barthel resultats'].std()

resultat = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultat)

Mitjana ± Desviació Estàndard del Barthel: 22.65 ± 24.63


In [65]:
Mitjana_barthel = AMB_PA_MECVV['Barthel resultats'].mean()
desviacio_estandard_barthel = AMB_PA_MECVV['Barthel resultats'].std()

resultat = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultat)

Mitjana ± Desviació Estàndard del Barthel: 13.75 ± 18.27


In [66]:
Mitjana_barthel = SENSE_PA['Barthel resultats'].mean()
desviacio_estandard_barthel = SENSE_PA['Barthel resultats'].std()

resultat = f"{Mitjana_barthel:.2f} ± {desviacio_estandard_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultat)

Mitjana ± Desviació Estàndard del Barthel: 53.12 ± 21.20


#### Pvalor

##### Normalitat

In [67]:
stat, p_value = shapiro(AMB_PA['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8517
Valor p = 0.0114
El nombre de files no segueix una distribució normal.


In [68]:
stat, p_value = shapiro(AMB_PA_MECVV['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8000
Valor p = 0.0286
El nombre de files no segueix una distribució normal.


In [69]:
stat, p_value = shapiro(SENSE_PA['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9475
Valor p = 0.6859
El nombre de files segueix una distribució normal.


In [70]:
stat, p_value = shapiro(TOTAL['Barthel resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8866
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [71]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'], AMB_PA_MECVV['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.47528505928431597


In [72]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'], SENSE_PA['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

P-valor entre AMB_PA i SENSE_PA: 0.00906817282777616


In [73]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'], SENSE_PA['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_13)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.00906817282777616


In [74]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_value_13)

P-valor entre AMB_PA i TOTAL: 0.00906817282777616


In [75]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL: 0.015133604112910346


In [76]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'], TOTAL['Barthel resultats'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

P-valor entre SENSE_PA i TOTAL: 0.6668885476033475


### Valors de dependència

##### AMB_PA

In [77]:
indepe_AMB_PA = (AMB_PA['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA: {indepe_AMB_PA:.2f}%")

Percentatge de indepe_AMB_PA: 0.00%


In [78]:
mod_AMB_PA = (AMB_PA['Barthel resultats'].between(61, 95)).mean() * 100
print(f"Percentatge de mod_AMB_PA: {mod_AMB_PA:.2f}%")

Percentatge de mod_AMB_PA: 5.88%


In [79]:
sev_AMB_PA = (AMB_PA['Barthel resultats'].between(21, 60)).mean() * 100
print(f"Percentatge de sev_AMB_PA: {sev_AMB_PA:.2f}%")

Percentatge de sev_AMB_PA: 41.18%


In [80]:
total_AMB_PA = (AMB_PA['Barthel resultats'] <= 20.0 ).mean() * 100
print(f"Percentatge de total_AMB_PA: {total_AMB_PA:.2f}%")

Percentatge de total_AMB_PA: 52.94%


##### AMB_PA_MECVV

In [81]:
indepe_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA_MECVV: {indepe_AMB_PA_MECVV:.2f}%")

Percentatge de indepe_AMB_PA_MECVV: 0.00%


In [82]:
mod_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'].between(61,95)).mean() * 100
print(f"Percentatge de mod_AMB_PA_MECVV: {mod_AMB_PA_MECVV:.2f}%")

Percentatge de mod_AMB_PA_MECVV: 0.00%


In [83]:
sev_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'].between(21,60)).mean() * 100
print(f"Percentatge de sev_AMB_PA_MECVV: {sev_AMB_PA_MECVV:.2f}%")

Percentatge de sev_AMB_PA_MECVV: 25.00%


In [84]:
total_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel resultats'] <= 20.0).mean() * 100
print(f"Percentatge de total_AMB_PA_MECVV: {total_AMB_PA_MECVV:.2f}%")

Percentatge de total_AMB_PA_MECVV: 75.00%


##### SENSE_PA

In [85]:
indepe_SENSE_PA = (SENSE_PA['Barthel resultats'] == 100.0).mean() * 100
print(f"Percentatge de indepe_SENSE_PA: {indepe_SENSE_PA:.2f}%")

Percentatge de indepe_SENSE_PA: 0.00%


In [86]:
mod_SENSE_PA = (SENSE_PA['Barthel resultats'].between(61,95)).mean() * 100
print(f"Percentatge de mod_SENSE_PA: {mod_SENSE_PA:.2f}%")

Percentatge de mod_SENSE_PA: 37.50%


In [87]:
sev_SENSE_PA = (SENSE_PA['Barthel resultats'].between(21,60)).mean() * 100
print(f"Percentatge de sev_SENSE_PA: {sev_SENSE_PA:.2f}%")

Percentatge de sev_SENSE_PA: 62.50%


In [88]:
total_SENSE_PA = (SENSE_PA['Barthel resultats'] <= 20).mean() * 100
print(f"Percentatge de total_SENSE_PA: {total_SENSE_PA:.2f}%")

Percentatge de total_SENSE_PA: 0.00%


#### Pvalor

In [89]:
##### Independència #TODO: subcategorias con chi cuadrado

In [90]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'] == 100.0, AMB_PA_MECVV['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV:", p_value_12)

P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV: 1.0


In [91]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'] == 100.0, SENSE_PA['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i indepe_SENSE_PA: 1.0


In [92]:
stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Barthel resultats'] == 100.0, SENSE_PA['Barthel resultats'] == 100.0, equal_var=False)
print("P-valor entre indepe_AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre indepe_AMB_PA_MECVV i SENSE_PA: 0.006056441109782507


In [93]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'] == 100.0, TOTAL['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_TOTAL:", p_value_14)

P-valor entre indepe_AMB_PA i indepe_TOTAL: 0.04855526544534558


In [94]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'] == 100.0, TOTAL['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA_MECVV i indepe_TOTAL:", p_value_24)

P-valor entre indepe_AMB_PA_MECVV i indepe_TOTAL: 0.17345845133549298


In [95]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'] == 100.0, TOTAL['Barthel resultats'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_SENSSE_PA i indepe_TOTAL:", p_value_34)

P-valor entre indepe_SENSSE_PA i indepe_TOTAL: 0.17345845133549298


##### Moderada

In [96]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'].between(61, 95), AMB_PA_MECVV['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV:", p_value_12)

P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV: 0.5483429354628137


In [97]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'].between(61, 95), SENSE_PA['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre indepe_AMB_PA i mod_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i mod_SENSE_PA: 0.05422099630917637


In [98]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'].between(61, 95), SENSE_PA['Barthel resultats'].between(61, 95), alternative='two-sided')  
print("P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA :", p_value_23)

P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA : 0.07458641569139825


In [99]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'].between(61, 95), TOTAL['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA i mod_TOTAL:", p_value_14)

P-valor entre mod_AMB_PA i mod_TOTAL: 0.23385439959965149


In [100]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'].between(61, 95), TOTAL['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA_MECVV i mod_TOTAL:", p_value_24)

P-valor entre mod_AMB_PA_MECVV i mod_TOTAL: 0.20185601492817817


In [101]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'].between(61, 95), TOTAL['Barthel resultats'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_SENSE_PA i mod_TOTAL:", p_value_34)

P-valor entre mod_SENSE_PA i mod_TOTAL: 0.21602478204322784


##### Severa

In [102]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'].between(21,60), AMB_PA_MECVV['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV:", p_value_12)

P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV: 0.4622499461255427


In [103]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'].between(21,60), SENSE_PA['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_SENSE_PA:", p_value_13)

P-valor entre sev_AMB_PA i sev_SENSE_PA: 0.3463247915940385


In [104]:
t_stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'].between(21,60), SENSE_PA['Barthel resultats'].between(21,60), alternative='two-sided' )
print("P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA:", p_value_23)

P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA: 0.16065967802771053


In [105]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'].between(21,60), TOTAL['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_TOTAL:", p_value_14)

P-valor entre sev_AMB_PA i sev_TOTAL: 0.31965398550389235


In [106]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'].between(21,60), TOTAL['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA_MECVV i sev_TOTAL:", p_value_24)

P-valor entre sev_AMB_PA_MECVV i sev_TOTAL: 0.8727371942854353


In [107]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'].between(21,60), TOTAL['Barthel resultats'].between(21,60), alternative='two-sided')
print("P-valor entre sev_SENSE_PA i sev_TOTAL:", p_value_34)

P-valor entre sev_SENSE_PA i sev_TOTAL: 0.05745304713254242


##### Total

In [108]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel resultats'] <= 20.0, AMB_PA_MECVV['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_AMB_PA_MECVV:", p_value_12)

P-valor entre total_AMB_PA i total_AMB_PA_MECVV: 0.31988618684011016


In [109]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel resultats'] <= 20.0, SENSE_PA['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_SENSE_PA:", p_value_13)

P-valor entre total_AMB_PA i total_SENSE_PA: 0.01293706233933995


In [110]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'] <= 20.0, SENSE_PA['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA_MECVV i total_SENSE_PA:", p_value_23)

P-valor entre total_AMB_PA_MECVV i total_SENSE_PA: 0.0033087017190950086


In [111]:
stat_14, p_value_14 = mannwhitneyu(AMB_PA['Barthel resultats'] <= 20.0, TOTAL['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_TOTAL:", p_value_14)

P-valor entre total_AMB_PA i total_TOTAL: 0.17208392420429652


In [112]:
stat_24, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Barthel resultats'] <= 20.0, TOTAL['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA_MECVV i total_TOTAL:", p_value_24)

P-valor entre total_AMB_PA_MECVV i total_TOTAL: 0.030106371453454194


In [113]:
stat_34, p_value_34 = mannwhitneyu(SENSE_PA['Barthel resultats'] <= 20.0, TOTAL['Barthel resultats'] <= 20.0, alternative='two-sided')
print("P-valor entre total_SENSE_PA i total_TOTAL:", p_value_34)

P-valor entre total_SENSE_PA i total_TOTAL: 0.0535696765403014


## Index del MNA

Primer convertiré la columna a tipus numèric ja que els valors que conté són de tipus objecte i amb aquest tipus no puc calcular la mitjana ni la desviació estàndard.

In [114]:
AMB_PA.loc[:, 'MNA resultats'] = pd.to_numeric(AMB_PA['MNA resultats'], errors='coerce')
AMB_PA_MECVV.loc[:, 'MNA resultats'] = pd.to_numeric(AMB_PA_MECVV['MNA resultats'], errors='coerce')
SENSE_PA.loc[:, 'MNA resultats'] = pd.to_numeric(SENSE_PA['MNA resultats'], errors='coerce')
TOTAL.loc[:, 'MNA resultats'] = pd.to_numeric(TOTAL['MNA resultats'], errors='coerce')

### Mitjana

In [115]:
Mitjana_mna = AMB_PA['MNA resultats'].mean()
desviacio_estandard_mna = AMB_PA['MNA resultats'].std()

resultat = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultat)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [116]:
Mitjana_mna = AMB_PA_MECVV['MNA resultats'].mean()
desviacio_estandard_mna = AMB_PA_MECVV['MNA resultats'].std()

resultat = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultat)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [117]:
Mitjana_mna = SENSE_PA['MNA resultats'].mean()
desviacio_estandard_mna = SENSE_PA['MNA resultats'].std()

resultat = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultat)

Mitjana ± Desviació Estàndard del mna: 0.00 ± 0.00


In [118]:
Mitjana_mna = TOTAL['MNA resultats'].mean()
desviacio_estandard_mna = TOTAL['MNA resultats'].std()

resultat = f"{Mitjana_mna:.2f} ± {desviacio_estandard_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultat)

Mitjana ± Desviació Estàndard del mna: 4.41 ± 7.20


#### Pvalor

##### Normalitat

In [119]:
stat, p_value = shapiro(AMB_PA['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [120]:
stat, p_value = shapiro(AMB_PA_MECVV['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [121]:
stat, p_value = shapiro(SENSE_PA['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [122]:
stat, p_value = shapiro(TOTAL['MNA resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [123]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['MNA resultats'], AMB_PA_MECVV['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [124]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['MNA resultats'], SENSE_PA['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [125]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['MNA resultats'], SENSE_PA['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [126]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [127]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [128]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['MNA resultats'], TOTAL['MNA resultats'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Valors de malnutrició

##### AMB_PA

In [129]:
norm_AMB_PA = AMB_PA['MNA resultats'].between(24, 30)
Percentatge_norm_AMB_PA = norm_AMB_PA.mean() * 100
num_observaciones_norm_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de norm_AMB_PA: {Percentatge_norm_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_norm_AMB_PA}")

Percentatge de norm_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [130]:
malnutr_AMB_PA = AMB_PA['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA = malnutr_AMB_PA.mean() * 100
num_observacions_malnutr_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de malnutr_AMB_PA: {Percentatge_malnutr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observacions_malnutr_AMB_PA}")

Percentatge de malnutr_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [131]:
malnut_AMB_PA = AMB_PA['MNA resultats'] < 17
Percentatge_malnut_AMB_PA = malnut_AMB_PA.mean() * 100
num_observacions_malnut_AMB_PA = malnut_AMB_PA.sum()

print(f"Percentatge de malnut_AMB_PA: {Percentatge_malnut_AMB_PA:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA: {num_observacions_malnut_AMB_PA}")

Percentatge de malnut_AMB_PA: 23.53%
Nombre d'observacions en malnut_AMB_PA: 4


##### AMB_PA_MECVV

In [132]:
norm_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'].between(24, 30)
Percentatge_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.mean() * 100
num_observacions_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de norm_AMB_PA_MECVV: {Percentatge_norm_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observacions_norm_AMB_PA_MECVV}")

Percentatge de norm_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [133]:
malnutr_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA_MECVV = malnutr_AMB_PA_MECVV.mean() * 100
num_observacions_malnutr_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de malnutr_AMB_PA_MECVV: {Percentatge_malnutr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observacions_malnutr_AMB_PA_MECVV}")

Percentatge de malnutr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [134]:
malnut_AMB_PA_MECVV = AMB_PA_MECVV['MNA resultats'] < 17
Percentatge_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.mean() * 100
num_observacions_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.sum()

print(f"Percentatge de malnut_AMB_PA_MECVV: {Percentatge_malnut_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA_MECVV: {num_observacions_malnut_AMB_PA_MECVV}")

Percentatge de malnut_AMB_PA_MECVV: 25.00%
Nombre d'observacions en malnut_AMB_PA_MECVV: 2


##### SENSE_PA

In [135]:
norm_SENSE_PA = SENSE_PA['MNA resultats'].between(24, 30)
Percentatge_norm_SENSE_PA = norm_SENSE_PA.mean() * 100
num_observacions_norm_SENSE_PA = norm_SENSE_PA.sum()

print(f"Percentatge de norm_SENSE_PA: {Percentatge_norm_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en norm_SENSE_PA: {num_observacions_norm_SENSE_PA}")

Percentatge de norm_SENSE_PA: 0.00%
Nombre d'observacions en norm_SENSE_PA: 0


In [136]:
malnutr_SENSE_PA = SENSE_PA['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_SENSE_PA = malnutr_SENSE_PA.mean() * 100
num_observacions_malnutr_SENSE_PA = malnutr_SENSE_PA.sum()

print(f"Percentatge de malnutr_SENSE_PA: {Percentatge_malnutr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnutr_SENSE_PA: {num_observacions_malnutr_SENSE_PA}")

Percentatge de malnutr_SENSE_PA: 0.00%
Nombre d'observacions en malnutr_SENSE_PA: 0


In [137]:
malnut_SENSE_PA = SENSE_PA['MNA resultats'] < 17
Percentatge_malnut_SENSE_PA = malnut_SENSE_PA.mean() * 100
num_observacions_malnut_SENSE_PA = malnut_SENSE_PA.sum()

print(f"Percentatge de malnut_SENSE_PA: {Percentatge_malnut_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnut_SENSE_PA: {num_observacions_malnut_SENSE_PA}")

Percentatge de malnut_SENSE_PA: 37.50%
Nombre d'observacions en malnut_SENSE_PA: 3


##### TOTAL

In [138]:
norm_TOTAL = TOTAL['MNA resultats'].between(24, 30)
Percentatge_norm_TOTAL = norm_TOTAL.mean() * 100
num_observacions_norm_TOTAL = norm_TOTAL.sum()

print(f"Percentatge de norm_TOTAL: {Percentatge_norm_TOTAL:.2f}%")
print(f"Nombre d'observacions en norm_TOTAL: {num_observacions_norm_TOTAL}")

Percentatge de norm_TOTAL: 0.00%
Nombre d'observacions en norm_TOTAL: 0


In [139]:
malnutr_TOTAL = TOTAL['MNA resultats'].between(17, 23.5)
Percentatge_malnutr_TOTAL = malnutr_TOTAL.mean() * 100
num_observacions_malnutr_TOTAL = malnutr_TOTAL.sum()

print(f"Percentatge de malnutr_TOTAL: {Percentatge_malnutr_TOTAL:.2f}%")
print(f"Nombre d'observacions en malnutr_TOTAL: {num_observacions_malnutr_TOTAL}")

Percentatge de malnutr_TOTAL: 4.00%
Nombre d'observacions en malnutr_TOTAL: 2


In [140]:
malnut_TOTAL = TOTAL['MNA resultats'] < 17
Percentatge_malnut_TOTAL = malnut_TOTAL.mean() * 100
num_observacions_malnut_TOTAL = malnut_TOTAL.sum()

print(f"Percentatge de malnut_TOTAL: {Percentatge_malnut_TOTAL:.2f}%")
print(f"Nombre d'observacions en malnut_TOTAL: {num_observacions_malnut_TOTAL}")

Percentatge de malnut_TOTAL: 18.00%
Nombre d'observacions en malnut_TOTAL: 9


#### Pvalor

##### Normal

In [141]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['MNA resultats'].between(24, 30), AMB_PA_MECVV['MNA resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV: nan


In [142]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['MNA resultats'].between(24, 30), SENSE_PA['MNA resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_AMB_PA i norm_SENSE_PA:", p_val_13)

P-valor entre norm_AMB_PA i norm_SENSE_PA: nan


In [143]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['MNA resultats'].between(24, 30), SENSE_PA['MNA resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA:", p_val_23)

P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA: nan


In [144]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['MNA resultats'].between(24, 30), TOTAL['MNA resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_AMB_PA i norm_TOTAL:", p_val_14)

P-valor entre norm_AMB_PA i norm_TOTAL: nan


In [145]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['MNA resultats'].between(24, 30), TOTAL['MNA resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_AMB_PA_MECVV i norm_TOTAL:", p_val_24)

P-valor entre norm_AMB_PA_MECVV i norm_TOTAL: nan


In [146]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['MNA resultats'].between(24, 30), TOTAL['MNA resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_SENSE_PA i norm_TOTAL:", p_val_34)

P-valor entre norm_SENSE_PA i norm_TOTAL: nan


##### Risc de malnutrició

In [147]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['MNA resultats'].between(17, 23.5), AMB_PA_MECVV['MNA resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV:", p_val_12)

P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV: nan


In [148]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['MNA resultats'].between(17, 23.5), SENSE_PA['MNA resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA:", p_val_13)

P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA: nan


In [149]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['MNA resultats'].between(17, 23.5), SENSE_PA['MNA resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA:", p_val_23)

P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA: nan


In [150]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['MNA resultats'].between(17, 23.5), TOTAL['MNA resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_AMB_PA i malnutr_TOTAL:", p_val_14)

P-valor entre malnutr_AMB_PA i malnutr_TOTAL: 0.15938605197461636


In [151]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['MNA resultats'].between(17, 23.5), TOTAL['MNA resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_AMB_PA_MECVVV i malnutr_TOTAL:", p_val_24)

P-valor entre malnutr_AMB_PA_MECVVV i malnutr_TOTAL: 0.15938605197461636


In [152]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['MNA resultats'].between(17, 23.5), TOTAL['MNA resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_SENSE_PA i malnutr_TOTAL:", p_val_34)

P-valor entre malnutr_SENSE_PA i malnutr_TOTAL: 0.15938605197461636


##### Malnutrició

In [153]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['MNA resultats'] <17 , 
                                AMB_PA_MECVV['MNA resultats'] < 17, equal_var=False)
print("P-valor entre malnut_AMB_PA i malnut_AMB_PA_MECVV:", p_val_12)

P-valor entre malnut_AMB_PA i malnut_AMB_PA_MECVV: 0.9410290647654455


In [154]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['MNA resultats'] <17 , 
                                SENSE_PA['MNA resultats'] < 17, equal_var=False)
print("P-valor entre malnut_AMB_PA i malnut_SENSE_PA:", p_val_13)

P-valor entre malnut_AMB_PA i malnut_SENSE_PA: 0.5214594319226733


In [155]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['MNA resultats'] <17 , 
                                SENSE_PA['MNA resultats'] < 17, equal_var=False)
print("P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA:", p_val_23)

P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA: 0.6186566171324552


In [156]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['MNA resultats'] <17 , 
                                TOTAL['MNA resultats'] < 17, equal_var=False)
print("P-valor entre malnut_AMB_PA i malnut_TOTAL:", p_val_14)

P-valor entre malnut_AMB_PA i malnut_TOTAL: 0.6472962045447526


In [157]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['MNA resultats'] <17 , 
                                TOTAL['MNA resultats'] < 17, equal_var=False)
print("P-valor entre malnut_AMB_PA_MECVV i malnut_TOTAL:", p_val_24)

P-valor entre malnut_AMB_PA_MECVV i malnut_TOTAL: 0.6949496714815766


In [158]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['MNA resultats'] <17 , 
                                TOTAL['MNA resultats'] < 17, equal_var=False)
print("P-valor entre malnut_SENSE_PA i malnut_TOTAL:", p_val_34)

P-valor entre malnut_SENSE_PA i malnut_TOTAL: 0.33617668089139807


## Index de l'EMINA

### Mitjana

In [159]:
Mitjana_emina = AMB_PA['EMINA sumatoris comparats'].mean()
desviacio_estandard_emina = AMB_PA['EMINA sumatoris comparats'].std()

resultat = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultat)

Mitjana ± Desviació Estàndard de l'emina: 7.94 ± 3.21


In [160]:
Mitjana_emina = AMB_PA_MECVV['EMINA sumatoris comparats'].mean()
desviacio_estandard_emina = AMB_PA_MECVV['EMINA sumatoris comparats'].std()

resultat = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultat)

Mitjana ± Desviació Estàndard de l'emina: 8.50 ± 1.51


In [161]:
Mitjana_emina = SENSE_PA['EMINA sumatoris comparats'].mean()
desviacio_estandard_emina = SENSE_PA['EMINA sumatoris comparats'].std()

resultat = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultat)

Mitjana ± Desviació Estàndard de l'emina: 7.00 ± 2.83


In [162]:
Mitjana_emina = TOTAL['EMINA sumatoris comparats'].mean()
desviacio_estandard_emina = TOTAL['EMINA sumatoris comparats'].std()

resultat = f"{Mitjana_emina:.2f} ± {desviacio_estandard_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultat)

Mitjana ± Desviació Estàndard de l'emina: 5.73 ± 4.08


#### Pvalor

##### Normalitat

In [163]:
stat, p_value = shapiro(AMB_PA['EMINA sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9770
Valor p = 0.9248
El nombre de files segueix una distribució normal.


In [164]:
stat, p_value = shapiro(AMB_PA_MECVV['EMINA sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9183
Valor p = 0.4158
El nombre de files segueix una distribució normal.


In [165]:
stat, p_value = shapiro(SENSE_PA['EMINA sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8758
Valor p = 0.1716
El nombre de files segueix una distribució normal.


In [166]:
stat, p_value = shapiro(TOTAL['EMINA sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [167]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA sumatoris comparats'], AMB_PA_MECVV['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.5598635957447045


In [168]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA sumatoris comparats'], SENSE_PA['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.46879803068245907


In [169]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'], SENSE_PA['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.21345611032992376


In [170]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['EMINA sumatoris comparats'], TOTAL['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [171]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'], TOTAL['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [172]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['EMINA sumatoris comparats'], TOTAL['EMINA sumatoris comparats'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Valors EMINA

##### AMB_PA

In [173]:
lowr_AMB_PA = AMB_PA['EMINA sumatoris comparats'].between(0, 5)
Percentatge_lowr_AMB_PA = lowr_AMB_PA.mean() * 100
num_observacions_lowr_AMB_PA = lowr_AMB_PA.sum()

print(f"Percentatge de lowr_AMB_PA: {Percentatge_lowr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA: {num_observacions_lowr_AMB_PA}")

Percentatge de lowr_AMB_PA: 17.65%
Nombre d'observacions en lowr_AMB_PA: 3


In [174]:
modr_AMB_PA = AMB_PA['EMINA sumatoris comparats'].between(6, 10)
Percentatge_modr_AMB_PA = modr_AMB_PA.mean() * 100
num_observacions_modr_AMB_PA = modr_AMB_PA.sum()

print(f"Percentatge de modr_AMB_PA: {Percentatge_modr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA: {num_observacions_modr_AMB_PA}")

Percentatge de modr_AMB_PA: 64.71%
Nombre d'observacions en modr_AMB_PA: 11


In [175]:
highr_AMB_PA = AMB_PA['EMINA sumatoris comparats'].between(11, 15)
Percentatge_highr_AMB_PA = highr_AMB_PA.mean() * 100
num_observacions_highr_AMB_PA = highr_AMB_PA.sum()

print(f"Percentatge de highr_AMB_PA: {Percentatge_highr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA: {num_observacions_highr_AMB_PA}")

Percentatge de highr_AMB_PA: 17.65%
Nombre d'observacions en highr_AMB_PA: 3


##### AMB_PA_MECVV

In [176]:
lowr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA sumatoris comparats'].between(0, 5)
Percentatge_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.mean() * 100
num_observacions_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.sum()

print(f"Percentatge de lowr_AMB_PA_MECVV: {Percentatge_lowr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA_MECVV: {num_observacions_lowr_AMB_PA_MECVV}")

Percentatge de lowr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en lowr_AMB_PA_MECVV: 0


In [177]:
modr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA sumatoris comparats'].between(6, 10)
Percentatge_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.mean() * 100
num_observacions_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.sum()

print(f"Percentatge de modr_AMB_PA_MECVV: {Percentatge_modr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA_MECVV: {num_observacions_modr_AMB_PA_MECVV}")

Percentatge de modr_AMB_PA_MECVV: 87.50%
Nombre d'observacions en modr_AMB_PA_MECVV: 7


In [178]:
highr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA sumatoris comparats'].between(11, 15)
Percentatge_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.mean() * 100
num_observacions_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.sum()

print(f"Percentatge de highr_AMB_PA_MECVV: {Percentatge_highr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA_MECVV: {num_observacions_highr_AMB_PA_MECVV}")

Percentatge de highr_AMB_PA_MECVV: 12.50%
Nombre d'observacions en highr_AMB_PA_MECVV: 1


##### SENSE_PA

In [179]:
lowr_SENSE_PA = SENSE_PA['EMINA sumatoris comparats'].between(0, 5)
Percentatge_lowr_SENSE_PA = lowr_SENSE_PA.mean() * 100
num_observacions_lowr_SENSE_PA = lowr_SENSE_PA.sum()

print(f"Percentatge de lowr_SENSE_PA: {Percentatge_lowr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en lowr_SENSE_PA: {num_observacions_lowr_SENSE_PA}")

Percentatge de lowr_SENSE_PA: 37.50%
Nombre d'observacions en lowr_SENSE_PA: 3


In [180]:
modr_SENSE_PA = SENSE_PA['EMINA sumatoris comparats'].between(6, 10)
Percentatge_modr_SENSE_PA = modr_SENSE_PA.mean() * 100
num_observacions_modr_SENSE_PA = modr_SENSE_PA.sum()

print(f"Percentatge de modr_SENSE_PA: {Percentatge_modr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modr_SENSE_PA: {num_observacions_modr_SENSE_PA}")

Percentatge de modr_SENSE_PA: 62.50%
Nombre d'observacions en modr_SENSE_PA: 5


In [181]:
highr_SENSE_PA = SENSE_PA['EMINA sumatoris comparats'].between(11, 15)
Percentatge_highr_SENSE_PA = highr_SENSE_PA.mean() * 100
num_observacions_highr_SENSE_PA = highr_SENSE_PA.sum()

print(f"Percentatge de highr_SENSE_PA: {Percentatge_highr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en highr_SENSE_PA: {num_observacions_highr_SENSE_PA}")

Percentatge de highr_SENSE_PA: 0.00%
Nombre d'observacions en highr_SENSE_PA: 0


##### TOTAL

In [182]:
lowr_TOTAL = TOTAL['EMINA sumatoris comparats'].between(0, 5)
Percentatge_lowr_TOTAL = lowr_TOTAL.mean() * 100
num_observacions_lowr_TOTAL = lowr_TOTAL.sum()

print(f"Percentatge de lowr_TOTAL: {Percentatge_lowr_TOTAL:.2f}%")
print(f"Nombre d'observacions en lowr_TOTAL: {num_observacions_lowr_TOTAL}")

Percentatge de lowr_TOTAL: 44.00%
Nombre d'observacions en lowr_TOTAL: 22


In [183]:
modr_TOTAL = TOTAL['EMINA sumatoris comparats'].between(6, 10)
Percentatge_modr_TOTAL = modr_TOTAL.mean() * 100
num_observacions_modr_TOTAL = modr_TOTAL.sum()

print(f"Percentatge de modr_TOTAL: {Percentatge_modr_TOTAL:.2f}%")
print(f"Nombre d'observacions en modr_TOTAL: {num_observacions_modr_TOTAL}")

Percentatge de modr_TOTAL: 42.00%
Nombre d'observacions en modr_TOTAL: 21


In [184]:
highr_TOTAL = TOTAL['EMINA sumatoris comparats'].between(11, 15)
Percentatge_highr_TOTAL = highr_TOTAL.mean() * 100
num_observacions_highr_TOTAL = highr_TOTAL.sum()

print(f"Percentatge de highr_TOTAL: {Percentatge_highr_TOTAL:.2f}%")
print(f"Nombre d'observacions en highr_TOTAL: {num_observacions_highr_TOTAL}")

Percentatge de highr_TOTAL: 10.00%
Nombre d'observacions en highr_TOTAL: 5


#### Pvalor 

##### Risc baix

In [185]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(0,5), AMB_PA_MECVV['EMINA sumatoris comparats'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.08262239198015026


In [186]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(0,5), SENSE_PA['EMINA sumatoris comparats'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.35666312890880325


In [187]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'].between(0,5), SENSE_PA['EMINA sumatoris comparats'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.07960201245519757


In [188]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(0,5), TOTAL['EMINA sumatoris comparats'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: 0.03308917948035935


In [189]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'].between(0,5), TOTAL['EMINA sumatoris comparats'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: 1.1304487497657605e-07


In [190]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['EMINA sumatoris comparats'].between(0,5), TOTAL['EMINA sumatoris comparats'].between(0,5), equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: 0.7478706542699041


##### Risc moderat

In [191]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(6,10), AMB_PA_MECVV['EMINA sumatoris comparats'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.20327534354395935


In [192]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(6,10), SENSE_PA['EMINA sumatoris comparats'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.9211148458207754


In [193]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'].between(6,10), SENSE_PA['EMINA sumatoris comparats'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.2806702345204662


In [194]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(6,10), TOTAL['EMINA sumatoris comparats'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: 0.1128840194536276


In [195]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'].between(6,10), TOTAL['EMINA sumatoris comparats'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: 0.008071365716514525


In [196]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['EMINA sumatoris comparats'].between(6,10), TOTAL['EMINA sumatoris comparats'].between(6,10), equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: 0.32251221712017597


##### Risc alt

In [197]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(11,15), AMB_PA_MECVV['EMINA sumatoris comparats'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7477825496543375


In [198]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(11,15), SENSE_PA['EMINA sumatoris comparats'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.08262239198015026


In [199]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'].between(11,15), SENSE_PA['EMINA sumatoris comparats'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.3506166628202075


In [200]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['EMINA sumatoris comparats'].between(11,15), TOTAL['EMINA sumatoris comparats'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: 0.47174454674638366


In [201]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['EMINA sumatoris comparats'].between(11,15), TOTAL['EMINA sumatoris comparats'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: 0.8542719807570616


In [202]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['EMINA sumatoris comparats'].between(11,15), TOTAL['EMINA sumatoris comparats'].between(11,15), equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: 0.02377846612391613


## Index de la canadenca (escala neurològica canadenca)

### Mitjana

In [203]:
Mitjana_canad = AMB_PA['Canadenca sumatoris comparats'].mean()
desviacio_estandard_canad = AMB_PA['Canadenca sumatoris comparats'].std()

resultat = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultat)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [204]:
Mitjana_canad = AMB_PA_MECVV['Canadenca sumatoris comparats'].mean()
desviacio_estandard_canad = AMB_PA_MECVV['Canadenca sumatoris comparats'].std()

resultat = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultat)

Mitjana ± Desviació Estàndard de la canadenca: nan ± nan


In [205]:
Mitjana_canad = SENSE_PA['Canadenca sumatoris comparats'].mean()
desviacio_estandard_canad = SENSE_PA['Canadenca sumatoris comparats'].std()

resultat = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultat)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [206]:
Mitjana_canad = TOTAL['Canadenca sumatoris comparats'].mean()
desviacio_estandard_canad = TOTAL['Canadenca sumatoris comparats'].std()

resultat = f"{Mitjana_canad:.2f} ± {desviacio_estandard_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultat)

Mitjana ± Desviació Estàndard de la canadenca: 8.00 ± 1.41


#### Pvalor

##### Normalitat

In [207]:
stat, p_value = shapiro(AMB_PA['Canadenca sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [208]:
stat, p_value = shapiro(AMB_PA_MECVV['Canadenca sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [209]:
stat, p_value = shapiro(SENSE_PA['Canadenca sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [210]:
stat, p_value = shapiro(TOTAL['Canadenca sumatoris comparats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [211]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'], AMB_PA_MECVV['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [212]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'], SENSE_PA['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [213]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'], SENSE_PA['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [214]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'], TOTAL['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [215]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'], TOTAL['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [216]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Canadenca sumatoris comparats'], TOTAL['Canadenca sumatoris comparats'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Valors canadenca

##### AMB_PA 

In [217]:
mild_AMB_PA = AMB_PA['Canadenca sumatoris comparats'].between(10.5, 11.5)
Percentatge_mild_AMB_PA = mild_AMB_PA.mean() * 100
num_observacions_mild_AMB_PA = mild_AMB_PA.sum()

print(f"Percentatge de mild_AMB_PA: {Percentatge_mild_AMB_PA:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA: {num_observacions_mild_AMB_PA}")

Percentatge de mild_AMB_PA: 0.00%
Nombre d'observacions en mild_AMB_PA: 0


In [218]:
modneu_AMB_PA = AMB_PA['Canadenca sumatoris comparats'].between(5,10)
Percentatge_modneu_AMB_PA = modneu_AMB_PA.mean() * 100
num_observacions_modneu_AMB_PA = modneu_AMB_PA.sum()

print(f"Percentatge de modneu_AMB_PA: {Percentatge_modneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA: {num_observacions_modneu_AMB_PA}")

Percentatge de modneu_AMB_PA: 5.88%
Nombre d'observacions en modneu_AMB_PA: 1


In [219]:
sevneu_AMB_PA = AMB_PA['Canadenca sumatoris comparats'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA = sevneu_AMB_PA.mean() * 100
num_observacions_sevneu_AMB_PA = sevneu_AMB_PA.sum()

print(f"Percentatge de sevneu_AMB_PA: {Percentatge_sevneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA: {num_observacions_sevneu_AMB_PA}")

Percentatge de sevneu_AMB_PA: 0.00%
Nombre d'observacions en sevneu_AMB_PA: 0


##### AMB_PA_MECVV

In [220]:
mild_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca sumatoris comparats'].between(10.5, 11.5)
Percentatge_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.mean() * 100
num_observacions_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.sum()

print(f"Percentatge de mild_AMB_PA_MECVV: {Percentatge_mild_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA_MECVV: {num_observacions_mild_AMB_PA_MECVV}")

Percentatge de mild_AMB_PA_MECVV: 0.00%
Nombre d'observacions en mild_AMB_PA_MECVV: 0


In [221]:
modneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca sumatoris comparats'].between(5,10)
Percentatge_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.mean() * 100
num_observacions_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.sum()

print(f"Percentatge de modneu_AMB_PA_MECVV: {Percentatge_modneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA_MECVV: {num_observacions_modneu_AMB_PA_MECVV}")

Percentatge de modneu_AMB_PA_MECVV: 0.00%
Nombre d'observacions en modneu_AMB_PA_MECVV: 0


In [222]:
sevneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca sumatoris comparats'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.mean() * 100
num_observacions_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.sum()

print(f"Percentatge de sevneu_AMB_PA_MECVV: {Percentatge_sevneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA_MECVV: {num_observacions_sevneu_AMB_PA_MECVV}")

Percentatge de sevneu_AMB_PA_MECVV: 0.00%
Nombre d'observacions en sevneu_AMB_PA_MECVV: 0


##### SENSE_PA

In [223]:
mild_SENSE_PA = SENSE_PA['Canadenca sumatoris comparats'].between(10.5, 11.5)
Percentatge_mild_SENSE_PA = mild_SENSE_PA.mean() * 100
num_observacions_mild_SENSE_PA = mild_SENSE_PA.sum()

print(f"Percentatge de mild_SENSE_PA: {Percentatge_mild_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en mild_SENSE_PA: {num_observacions_mild_SENSE_PA}")

Percentatge de mild_SENSE_PA: 0.00%
Nombre d'observacions en mild_SENSE_PA: 0


In [224]:
modneu_SENSE_PA = SENSE_PA['Canadenca sumatoris comparats'].between(5,10)
Percentatge_modneu_SENSE_PA = modneu_SENSE_PA.mean() * 100
num_observacions_modneu_SENSE_PA = modneu_SENSE_PA.sum()

print(f"Percentatge de modneu_SENSE_PA: {Percentatge_modneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modneu_SENSE_PA: {num_observacions_modneu_SENSE_PA}")

Percentatge de modneu_SENSE_PA: 12.50%
Nombre d'observacions en modneu_SENSE_PA: 1


In [225]:
sevneu_SENSE_PA = SENSE_PA['Canadenca sumatoris comparats'].between(1.5, 4.5)
Percentatge_sevneu_SENSE_PA = sevneu_SENSE_PA.mean() * 100
num_observacions_sevneu_SENSE_PA = sevneu_SENSE_PA.sum()

print(f"Percentatge de sevneu_SENSE_PA: {Percentatge_sevneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_SENSE_PA: {num_observacions_sevneu_SENSE_PA}")

Percentatge de sevneu_SENSE_PA: 0.00%
Nombre d'observacions en sevneu_SENSE_PA: 0


##### TOTAL

In [226]:
mild_TOTAL = TOTAL['Canadenca sumatoris comparats'].between(10.5, 11.5)
Percentatge_mild_TOTAL = mild_TOTAL.mean() * 100
num_observacions_mild_TOTAL = mild_TOTAL.sum()

print(f"Percentatge de mild_TOTAL: {Percentatge_mild_TOTAL:.2f}%")
print(f"Nombre d'observacions en mild_TOTAL: {num_observacions_mild_TOTAL}")

Percentatge de mild_TOTAL: 0.00%
Nombre d'observacions en mild_TOTAL: 0


In [227]:
modneu_TOTAL = TOTAL['Canadenca sumatoris comparats'].between(5,10)
Percentatge_modneu_TOTAL = modneu_TOTAL.mean() * 100
num_observacions_modneu_TOTAL = modneu_TOTAL.sum()

print(f"Percentatge de modneu_TOTAL: {Percentatge_modneu_TOTAL:.2f}%")
print(f"Nombre d'observacions en modneu_TOTAL: {num_observacions_modneu_TOTAL}")

Percentatge de modneu_TOTAL: 4.00%
Nombre d'observacions en modneu_TOTAL: 2


In [228]:
sevneu_TOTAL = TOTAL['Canadenca sumatoris comparats'].between(1.5, 4.5)
Percentatge_sevneu_TOTAL = sevneu_TOTAL.mean() * 100
num_observacions_sevneu_TOTAL = sevneu_TOTAL.sum()

print(f"Percentatge de sevneu_TOTAL: {Percentatge_sevneu_TOTAL:.2f}%")
print(f"Nombre d'observacions en sevneu_TOTAL: {num_observacions_sevneu_TOTAL}")

Percentatge de sevneu_TOTAL: 0.00%
Nombre d'observacions en sevneu_TOTAL: 0


#### Pvalor

##### Dèficit neurològic lleuger (mild)

In [229]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(10.5,11.5), AMB_PA_MECVV['Canadenca sumatoris comparats'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [230]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(10.5,11.5), SENSE_PA['Canadenca sumatoris comparats'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [231]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'].between(10.5,11.5), SENSE_PA['Canadenca sumatoris comparats'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [232]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(10.5,11.5), TOTAL['Canadenca sumatoris comparats'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [233]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'].between(10.5,11.5), TOTAL['Canadenca sumatoris comparats'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [234]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Canadenca sumatoris comparats'].between(10.5,11.5), TOTAL['Canadenca sumatoris comparats'].between(10.5,11.5), equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


##### Dèficit neurològic moderat

In [235]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(5,10), AMB_PA_MECVV['Canadenca sumatoris comparats'].between(5, 10), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.3321949846529795


In [236]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(5,10), SENSE_PA['Canadenca sumatoris comparats'].between(5, 10), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.6420034888551491


In [237]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'].between(5,10), SENSE_PA['Canadenca sumatoris comparats'].between(5, 10), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.3506166628202075


In [238]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(5,10), TOTAL['Canadenca sumatoris comparats'].between(5, 10), equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: 0.7751366695188593


In [239]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'].between(5,10), TOTAL['Canadenca sumatoris comparats'].between(5, 10), equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: 0.15938605197461647


In [240]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Canadenca sumatoris comparats'].between(5,10), TOTAL['Canadenca sumatoris comparats'].between(5, 10), equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: 0.5262919030230506


##### Dèficit neurològic greu (sever)

In [241]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(1.5,4.5), AMB_PA_MECVV['Canadenca sumatoris comparats'].between(1.5,4.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [242]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(1.5, 4.5), SENSE_PA['Canadenca sumatoris comparats'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [243]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'].between(1.5,4.5), SENSE_PA['Canadenca sumatoris comparats'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [244]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Canadenca sumatoris comparats'].between(1.5,4.5), TOTAL['Canadenca sumatoris comparats'].between(1.5,4.5), equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [245]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Canadenca sumatoris comparats'].between(1.5,4.5), TOTAL['Canadenca sumatoris comparats'].between(1.5,4.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [246]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Canadenca sumatoris comparats'].between(1.5,4.5), TOTAL['Canadenca sumatoris comparats'].between(1.5,4.5), equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


## Infeccions respiratòries

##### AMB_PA 

In [247]:
mainresp_AMB_PA= (AMB_PA['Infeccions respiratòries principals diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 41.18%


In [248]:
LRTI_AMB_PA= (AMB_PA['LRTI diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA:.2f}%")

Percentatge de LRTI: 17.65%


In [249]:
COPD_AMB_PA= (AMB_PA['Exacerbacions de COPD diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA:.2f}%")

Percentatge de exacerbacions de COPD: 29.41%


In [250]:
fibro_AMB_PA= (AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### AMB_PA_MECVV

In [251]:
mainresp_AMB_PA_MECVV= (AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA_MECVV:.2f}%")

Percentatge de infeccions respiratòries principals : 50.00%


In [252]:
LRTI_AMB_PA_MECVV= (AMB_PA_MECVV['LRTI diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA_MECVV:.2f}%")

Percentatge de LRTI: 12.50%


In [253]:
COPD_AMB_PA_MECVV = (AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA_MECVV:.2f}%")

Percentatge de exacerbacions de COPD: 50.00%


In [254]:
fibro_AMB_PA_MECVV = (AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA_MECVV:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### SENSE_PA

In [255]:
mainresp_SENSE_PA= (SENSE_PA['Infeccions respiratòries principals diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_SENSE_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 50.00%


In [256]:
LRTI_SENSE_PA= (SENSE_PA['LRTI diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_SENSE_PA:.2f}%")

Percentatge de LRTI: 25.00%


In [257]:
COPD_SENSE_PA= (SENSE_PA['Exacerbacions de COPD diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_SENSE_PA:.2f}%")

Percentatge de exacerbacions de COPD: 25.00%


In [258]:
fibro_SENSE_PA= (SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_SENSE_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


#### Pvalor

##### Normalitat

In [259]:
stat, p_value = shapiro(AMB_PA['Infeccions respiratòries principals diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6317
Valor p = 0.00002144
El nombre de files no segueix una distribució normal.


In [260]:
stat, p_value = shapiro(AMB_PA['LRTI diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4702
Valor p = 0.00000073
El nombre de files no segueix una distribució normal.


In [261]:
stat, p_value = shapiro(AMB_PA['Exacerbacions de COPD diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5785
Valor p = 0.00000650
El nombre de files no segueix una distribució normal.


In [262]:
stat, p_value = shapiro(AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [263]:
stat, p_value = shapiro(AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.00089170
El nombre de files no segueix una distribució normal.


In [264]:
stat, p_value = shapiro(AMB_PA_MECVV['LRTI diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4184
Valor p = 0.00000105
El nombre de files no segueix una distribució normal.


In [265]:
stat, p_value = shapiro(AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.00089170
El nombre de files no segueix una distribució normal.


In [266]:
stat, p_value = shapiro(AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


In [267]:
stat, p_value = shapiro(SENSE_PA['Infeccions respiratòries principals diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6647
Valor p = 0.00089170
El nombre de files no segueix una distribució normal.


In [268]:
stat, p_value = shapiro(SENSE_PA['LRTI diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5659
Valor p = 0.00006323
El nombre de files no segueix una distribució normal.


In [269]:
stat, p_value = shapiro(SENSE_PA['Exacerbacions de COPD diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5659
Valor p = 0.00006323
El nombre de files no segueix una distribució normal.


In [270]:
stat, p_value = shapiro(SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


In [271]:
stat, p_value = shapiro(TOTAL['Infeccions respiratòries principals diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6275
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [272]:
stat, p_value = shapiro(TOTAL['LRTI diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5113
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [273]:
stat, p_value = shapiro(TOTAL['Exacerbacions de COPD diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5879
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [274]:
stat, p_value = shapiro(TOTAL['Fibrosi pulmonar i fibrotòrax diagnosticades'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


##### Infeccions respiratòries principals 

In [275]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Infeccions respiratòries principals diagnosticades'], columns=AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.


In [276]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Infeccions respiratòries principals diagnosticades'], columns=SENSE_PA['Infeccions respiratòries principals diagnosticades'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i SENSE_PA.


In [277]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'], columns=SENSE_PA['Infeccions respiratòries principals diagnosticades'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i AMB_PA_MECVV.")

ValueError: No data; `observed` has size 0.

In [278]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Infeccions respiratòries principals diagnosticades'], columns=TOTAL['Infeccions respiratòries principals diagnosticades'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_14)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 13.122091836734691
P-valor entre AMB_PA i AMB_PA_MECVV: 0.0002918340067889046
Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA i TOTAL.


In [279]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Infeccions respiratòries principals diagnosticades'], columns=TOTAL['Infeccions respiratòries principals diagnosticades'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en AMB_PA_MECVV i TOTAL.


In [280]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Infeccions respiratòries principals diagnosticades'], columns=TOTAL['Infeccions respiratòries principals diagnosticades'])

chi2_34, p_value_34, dof_4, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre AMB_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'infeccions respiratòries en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.5
P-valor entre AMB_PA i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions d'infeccions respiratòries en SENSE_PA i TOTAL.


##### LRTI

In [281]:
contingency_table_12 = pd.crosstab(index=AMB_PA['LRTI diagnosticada'], columns=AMB_PA_MECVV['LRTI diagnosticada'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.9600000000000002
P-valor entre AMB_PA i AMB_PA_MECVV: 0.32718687779030275
No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i AMB_PA_MECVV.


In [282]:
contingency_table_13 = pd.crosstab(index=AMB_PA['LRTI diagnosticada'], columns=SENSE_PA['LRTI diagnosticada'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i SENSE_PA.


In [283]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['LRTI diagnosticada'], columns=SENSE_PA['LRTI diagnosticada'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [284]:
contingency_table_14 = pd.crosstab(index=AMB_PA['LRTI diagnosticada'], columns=TOTAL['LRTI diagnosticada'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 10.815334467120184
P-valor entre AMB_PA i TOTAL: 0.0010066283741798992
Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA i TOTAL.


In [285]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['LRTI diagnosticada'], columns=TOTAL['LRTI diagnosticada'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de LRTI en AMB_PA_MECVV i TOTAL.


In [286]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['LRTI diagnosticada'], columns=TOTAL['LRTI diagnosticada'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SNESE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de LRTI en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de LRTI en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 3.5555555555555554
P-valor entre SNESE_PA i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de LRTI en SENSE_PA i TOTAL.


##### Exacerbacions de COPD

In [287]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Exacerbacions de COPD diagnosticades'], columns=AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i AMB_PA_MECV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i AMB_PA_MECVV.


In [288]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Exacerbacions de COPD diagnosticades'], columns=SENSE_PA['Exacerbacions de COPD diagnosticades'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i SENSE_PA.


In [289]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'], columns=SENSE_PA['Exacerbacions de COPD diagnosticades'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [290]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Exacerbacions de COPD diagnosticades'], columns=TOTAL['Exacerbacions de COPD diagnosticades'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA i TOTAL.


In [291]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Exacerbacions de COPD diagnosticades'], columns=TOTAL['Exacerbacions de COPD diagnosticades'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en AMB_PA_MECVV i TOTAL.


In [292]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Exacerbacions de COPD diagnosticades'], columns=TOTAL['Exacerbacions de COPD diagnosticades'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 3.5555555555555554
P-valor entre SENSE_PA i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions d'exacerbacions de COPD en SENSE_PA i TOTAL.


##### Fibrosi pulmonar i fibrotòrax

In [293]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i AMB_PA_MECVV.


In [294]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i SENSE_PA.


In [295]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [296]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=TOTAL['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA i TOTAL.


In [297]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=TOTAL['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en AMB_PA_MECVV i TOTAL.


In [298]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Fibrosi pulmonar i fibrotòrax diagnosticades'], columns=TOTAL['Fibrosi pulmonar i fibrotòrax diagnosticades'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de fibrosi pulmonar i fibrotòrax en SENSE_PA i TOTAL.


## Resultats de laboratori

Les columnes generades a partir d'aquests resultats contenen valors tipus objecte, per tant, primer els transformaré a tipus numèric per poder realitzar les operacions que neccessitaré fer.

### Albúmina

In [299]:
AMB_PA.loc[:, 'Albúmina'] = pd.to_numeric(AMB_PA['Albúmina'], errors='coerce').dropna()

Mitjana_alb = AMB_PA['Albúmina'].mean()
desviacio_estandard_alb = AMB_PA['Albúmina'].std()

resultat = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de l'albúmina:", resultat)

Mitjana ± Desviació Estàndard de l'albúmina: 3.34 ± 0.38


In [300]:
AMB_PA_MECVV.loc[:, 'Albúmina'] = pd.to_numeric(AMB_PA_MECVV['Albúmina'], errors='coerce').dropna()

Mitjana_alb = AMB_PA_MECVV['Albúmina'].mean()
desviacio_estandard_alb = AMB_PA_MECVV['Albúmina'].std()

resultat = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de l'albúmina:", resultat)

Mitjana ± Desviació Estàndard de l'albúmina: 3.33 ± 0.38


In [301]:
SENSE_PA.loc[:, 'Albúmina'] = pd.to_numeric(SENSE_PA['Albúmina'], errors='coerce').dropna()

Mitjana_alb = SENSE_PA['Albúmina'].mean()
desviacio_estandard_alb = SENSE_PA['Albúmina'].std()

resultat = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de l'albúmina:", resultat)

Mitjana ± Desviació Estàndard de l'albúmina: 3.55 ± 0.37


In [302]:
TOTAL.loc[:, 'Albúmina'] = pd.to_numeric(TOTAL['Albúmina'], errors='coerce').dropna()

Mitjana_alb = TOTAL['Albúmina'].mean()
desviacio_estandard_alb = TOTAL['Albúmina'].std()

resultat = f"{Mitjana_alb:.2f} ± {desviacio_estandard_alb:.2f}"
print("Mitjana ± Desviació Estàndard de l'albúmina:", resultat)

Mitjana ± Desviació Estàndard de l'albúmina: 3.55 ± 0.64


#### Pvalor

##### Normalitat

In [303]:
albpa = AMB_PA['Albúmina']
stat, p_value = shapiro(albpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [304]:
albAMBpamecvv = AMB_PA_MECVV['Albúmina']
stat, p_value = shapiro(albAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [305]:
albsensepa= SENSE_PA['Albúmina']
stat, p_value = shapiro(albsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [306]:
albtotal = TOTAL['Albúmina']
stat, p_value = shapiro(albtotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [307]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Albúmina'], AMB_PA_MECVV['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [308]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Albúmina'], SENSE_PA['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [309]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Albúmina'], SENSE_PA['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [310]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Albúmina'], TOTAL['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [311]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Albúmina'], TOTAL['Albúmina'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [312]:
t_stat_34, p_val_14 = ttest_ind(SENSE_PA['Albúmina'], TOTAL['Albúmina'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Proteïnes totals

In [313]:
AMB_PA.loc[:, 'Proteïnes totals'] = pd.to_numeric(AMB_PA['Proteïnes totals'], errors='coerce')

Mitjana_prot = AMB_PA['Proteïnes totals'].mean()
desviacio_estandard_prot = AMB_PA['Proteïnes totals'].std()

resultat = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de les proteïnes totals:", resultat)

Mitjana ± Desviació Estàndard de les proteïnes totals: 6.03 ± 0.73


In [314]:
AMB_PA_MECVV.loc[:, 'Proteïnes totals'] = pd.to_numeric(AMB_PA_MECVV['Proteïnes totals'], errors='coerce')

Mitjana_prot = AMB_PA_MECVV['Proteïnes totals'].mean()
desviacio_estandard_prot = AMB_PA_MECVV['Proteïnes totals'].std()

resultat = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de les proteïnes totals:", resultat)

Mitjana ± Desviació Estàndard de les proteïnes totals: 6.00 ± 0.53


In [315]:
SENSE_PA.loc[:, 'Proteïnes totals'] = pd.to_numeric(SENSE_PA['Proteïnes totals'], errors='coerce')

Mitjana_prot = SENSE_PA['Proteïnes totals'].mean()
desviacio_estandard_prot = SENSE_PA['Proteïnes totals'].std()

resultat = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de les proteïnes totals:", resultat)

Mitjana ± Desviació Estàndard de les proteïnes totals: 6.30 ± 0.80


In [316]:
TOTAL.loc[:, 'Proteïnes totals'] = pd.to_numeric(TOTAL['Proteïnes totals'], errors='coerce')

Mitjana_prot = TOTAL['Proteïnes totals'].mean()
desviacio_estandard_prot = TOTAL['Proteïnes totals'].std()

resultat = f"{Mitjana_prot:.2f} ± {desviacio_estandard_prot:.2f}"
print("Mitjana ± Desviació Estàndard de les proteïnes totals:", resultat)

Mitjana ± Desviació Estàndard de les proteïnes totals: 6.07 ± 0.74


#### Pvalor

##### Normalitat

In [317]:
protAMBpa=AMB_PA['Proteïnes totals']
stat, p_value = shapiro(protAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [318]:
protAMBpamecvv=AMB_PA_MECVV['Proteïnes totals']
stat, p_value = shapiro(protAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [319]:
protsensepa=SENSE_PA['Proteïnes totals']
stat, p_value = shapiro(protsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [320]:
prottotal=TOTAL['Proteïnes totals']
stat, p_value = shapiro(prottotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [321]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Proteïnes totals'], AMB_PA_MECVV['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [322]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Proteïnes totals'], SENSE_PA['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [323]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Proteïnes totals'], SENSE_PA['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [324]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Proteïnes totals'], TOTAL['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [325]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Proteïnes totals'], TOTAL['Proteïnes totals'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [326]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Proteïnes totals'], TOTAL['Proteïnes totals'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Hemoglobina

In [327]:
AMB_PA.loc[:, 'Hb'] = pd.to_numeric(AMB_PA['Hb'], errors='coerce')

Mitjana_hb = AMB_PA['Hb'].mean()
desviacio_estandard_hb = AMB_PA['Hb'].std()

resultat = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de l'hemoglobina:", resultat)

Mitjana ± Desviació Estàndard de l'hemoglobina: 12.34 ± 1.77


In [328]:
AMB_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(AMB_PA_MECVV['Hb'], errors='coerce')

Mitjana_hb = AMB_PA_MECVV['Hb'].mean()
desviacio_estandard_hb = AMB_PA_MECVV['Hb'].std()

resultat = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de l'hemoglobina:", resultat)

Mitjana ± Desviació Estàndard de l'hemoglobina: 11.77 ± 2.58


In [329]:
SENSE_PA.loc[:, 'Hb'] = pd.to_numeric(SENSE_PA['Hb'], errors='coerce')

Mitjana_hb = SENSE_PA['Hb'].mean()
desviacio_estandard_hb = SENSE_PA['Hb'].std()

resultat = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de l'hemoglobina:", resultat)

Mitjana ± Desviació Estàndard de l'hemoglobina: 12.83 ± 1.21


In [330]:
TOTAL.loc[:, 'Hb'] = pd.to_numeric(TOTAL['Hb'], errors='coerce')

Mitjana_hb = TOTAL['Hb'].mean()
desviacio_estandard_hb = TOTAL['Hb'].std()

resultat = f"{Mitjana_hb:.2f} ± {desviacio_estandard_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultat)

Mitjana ± Desviació Estàndard de la hemoglobina: 13.21 ± 2.11


##### Pvalor

##### Normalitat

In [331]:
hbAMBpa=AMB_PA['Hb'].dropna()
stat, p_value = shapiro(hbAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9304
Valor p = 0.48501101
El nombre de files segueix una distribució normal.


In [332]:
hbAMBpamecvv=AMB_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8288
Valor p = 0.18525796
El nombre de files segueix una distribució normal.


In [333]:
hbsensepa=SENSE_PA['Hb'].dropna()
stat, p_value = shapiro(hbsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9762
Valor p = 0.93094253
El nombre de files segueix una distribució normal.


In [334]:
hbtotal=TOTAL['Hb'].dropna()
stat, p_value = shapiro(hbtotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9520
Valor p = 0.29975140
El nombre de files segueix una distribució normal.


In [335]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Hb'], AMB_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [336]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [337]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [338]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Hb'], TOTAL['Hb'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [339]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Hb'], TOTAL['Hb'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [340]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Hb'], TOTAL['Hb'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Colesterol 

In [341]:
AMB_PA.loc[:, 'Colesterol total'] = pd.to_numeric(AMB_PA['Colesterol total'], errors='coerce')

Mitjana_col = AMB_PA['Colesterol total'].mean()
desviacio_estandard_col = AMB_PA['Colesterol total'].std()

resultat = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultat)

Mitjana ± Desviació Estàndard del colesterol total: 132.10 ± 34.83


In [342]:
AMB_PA_MECVV.loc[:, 'Colesterol total'] = pd.to_numeric(AMB_PA_MECVV['Colesterol total'], errors='coerce')

Mitjana_col = AMB_PA_MECVV['Colesterol total'].mean()
desviacio_estandard_col = AMB_PA_MECVV['Colesterol total'].std()

resultat = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultat)

Mitjana ± Desviació Estàndard del colesterol total: 151.47 ± 33.55


In [343]:
SENSE_PA.loc[:, 'Colesterol total'] = pd.to_numeric(SENSE_PA['Colesterol total'], errors='coerce')

Mitjana_col = SENSE_PA['Colesterol total'].mean()
desviacio_estandard_col = SENSE_PA['Colesterol total'].std()

resultat = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultat)

Mitjana ± Desviació Estàndard del colesterol total: 115.28 ± 23.69


In [344]:
TOTAL.loc[:, 'Colesterol total'] = pd.to_numeric(TOTAL['Colesterol total'], errors='coerce')

Mitjana_col = TOTAL['Colesterol total'].mean()
desviacio_estandard_col = TOTAL['Colesterol total'].std()

resultat = f"{Mitjana_col:.2f} ± {desviacio_estandard_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultat)

Mitjana ± Desviació Estàndard del colesterol total: 142.67 ± 40.92


#### Pvalor

##### Normalitat 



In [345]:
colAMBpa=AMB_PA['Colesterol total'].dropna()
stat, p_value = shapiro(colAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9887
Valor p = 0.99048226
El nombre de files segueix una distribució normal.


In [346]:
colAMBpamecvv=AMB_PA_MECVV['Colesterol total'].dropna()
stat, p_value = shapiro(colAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9941
Valor p = 0.85359605
El nombre de files segueix una distribució normal.


In [347]:
colsensepa=SENSE_PA['Colesterol total'].dropna()
stat, p_value = shapiro(colsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9231
Valor p = 0.55007868
El nombre de files segueix una distribució normal.


In [348]:
coltotal=TOTAL['Colesterol total'].dropna()
stat, p_value = shapiro(coltotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9629
Valor p = 0.63113194
El nombre de files segueix una distribució normal.


In [349]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Colesterol total'], AMB_PA_MECVV['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [350]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Colesterol total'], SENSE_PA['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [351]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Colesterol total'], SENSE_PA['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [352]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Colesterol total'], TOTAL['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [353]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Colesterol total'], TOTAL['Colesterol total'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [354]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Colesterol total'], TOTAL['Colesterol total'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Leucòcits

In [355]:
AMB_PA.loc[:, 'Leucos'] = pd.to_numeric(AMB_PA['Leucos'], errors='coerce')

Mitjana_leu = AMB_PA['Leucos'].mean()
desviacio_estandard_leu = AMB_PA['Leucos'].std()

resultat = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultat)

Mitjana ± Desviació Estàndard dels leucòcits: 12.69 ± 4.88


In [356]:
AMB_PA_MECVV.loc[:, 'Leucos'] = pd.to_numeric(AMB_PA_MECVV['Leucos'], errors='coerce')

Mitjana_leu = AMB_PA_MECVV['Leucos'].mean()
desviacio_estandard_leu = AMB_PA_MECVV['Leucos'].std()

resultat = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultat)

Mitjana ± Desviació Estàndard dels leucòcits: 8.83 ± 0.70


In [357]:
SENSE_PA.loc[:, 'Leucos'] = pd.to_numeric(SENSE_PA['Leucos'], errors='coerce')

Mitjana_leu = SENSE_PA['Leucos'].mean()
desviacio_estandard_leu = SENSE_PA['Leucos'].std()

resultat = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultat)

Mitjana ± Desviació Estàndard dels leucòcits: 10.68 ± 6.49


In [358]:
TOTAL.loc[:, 'Leucos'] = pd.to_numeric(TOTAL['Leucos'], errors='coerce')

Mitjana_leu = TOTAL['Leucos'].mean()
desviacio_estandard_leu = TOTAL['Leucos'].std()

resultat = f"{Mitjana_leu:.2f} ± {desviacio_estandard_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultat)

Mitjana ± Desviació Estàndard dels leucòcits: 11.34 ± 6.43


#### Pvalor 

##### Normalitat

In [359]:
leuambpa=AMB_PA['Leucos'].dropna()
stat, p_value = shapiro(leuambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8482
Valor p = 0.09127190
El nombre de files segueix una distribució normal.


In [360]:
leuambpamecvv=AMB_PA_MECVV['Leucos'].dropna()
stat, p_value = shapiro(leuambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8811
Valor p = 0.32754766
El nombre de files segueix una distribució normal.


In [361]:
leusensepa=SENSE_PA['Leucos'].dropna()
stat, p_value = shapiro(leusensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8233
Valor p = 0.09427563
El nombre de files segueix una distribució normal.


In [362]:
leutotal=TOTAL['Leucos'].dropna()
stat, p_value = shapiro(leutotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8070
Valor p = 0.00049781
El nombre de files no segueix una distribució normal.


In [363]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Leucos'], AMB_PA_MECVV['Leucos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [364]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Leucos'], SENSE_PA['Leucos'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [365]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Leucos'], SENSE_PA['Leucos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [366]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['Leucos'], TOTAL['Leucos'], equal_var=False)
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [367]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Leucos'], TOTAL['Leucos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [368]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['Leucos'], TOTAL['Leucos'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Limfòcits %

In [369]:
AMB_PA.loc[:, 'Limfos'] = pd.to_numeric(AMB_PA['Limfos'], errors='coerce')

Mitjana_limf = AMB_PA['Limfos'].mean()
desviacio_estandard_limf = AMB_PA['Limfos'].std()

resultat = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultat)

Mitjana ± Desviació Estàndard dels limfòcits %: 16.70 ± 12.29


In [370]:
AMB_PA_MECVV.loc[:, 'Limfos'] = pd.to_numeric(AMB_PA_MECVV['Limfos'], errors='coerce')

Mitjana_limf = AMB_PA_MECVV['Limfos'].mean()
desviacio_estandard_limf = AMB_PA_MECVV['Limfos'].std()

resultat = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultat)

Mitjana ± Desviació Estàndard dels limfòcits %: 9.57 ± 2.10


In [371]:
SENSE_PA.loc[:, 'Limfos'] = pd.to_numeric(SENSE_PA['Limfos'], errors='coerce')

Mitjana_limf = SENSE_PA['Limfos'].mean()
desviacio_estandard_limf = SENSE_PA['Limfos'].std()

resultat = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultat)

Mitjana ± Desviació Estàndard dels limfòcits %: 26.66 ± 9.21


In [372]:
TOTAL.loc[:, 'Limfos'] = pd.to_numeric(TOTAL['Limfos'], errors='coerce')

Mitjana_limf = TOTAL['Limfos'].mean()
desviacio_estandard_limf = TOTAL['Limfos'].std()

resultat = f"{Mitjana_limf:.2f} ± {desviacio_estandard_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultat)

Mitjana ± Desviació Estàndard dels limfòcits %: 19.33 ± 11.32


#### Pvalor

##### Normalitat

In [373]:
limfambpa=AMB_PA['Limfos'].dropna()
stat, p_value = shapiro(limfambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8208
Valor p = 0.04757269
El nombre de files no segueix una distribució normal.


In [374]:
limfambpamecvv=AMB_PA_MECVV['Limfos'].dropna()
stat, p_value = shapiro(limfambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9992
Valor p = 0.94750571
El nombre de files segueix una distribució normal.


In [375]:
limfsensepa=SENSE_PA['Limfos'].dropna()
stat, p_value = shapiro(limfsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9815
Valor p = 0.94260450
El nombre de files segueix una distribució normal.


In [376]:
limftotal=TOTAL['Limfos'].dropna()
stat, p_value = shapiro(limftotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9283
Valor p = 0.11315474
El nombre de files segueix una distribució normal.


In [377]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Limfos'], AMB_PA_MECVV['Limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [378]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Limfos'], SENSE_PA['Limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [379]:
stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Limfos'], SENSE_PA['Limfos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [380]:
stat_14, p_val_14 = mannwhitneyu(AMB_PA['Limfos'], TOTAL['Limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [381]:
stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['Limfos'], TOTAL['Limfos'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [382]:
stat_34, p_val_34 = ttest_ind(SENSE_PA['Limfos'], TOTAL['Limfos'], equal_var=False)
print("P-valor entre SNESE_PA i TOTAL:", p_val_14)

P-valor entre SNESE_PA i TOTAL: nan


### Proteïna C Reactiva

In [383]:
AMB_PA.loc[:, 'Prot C react'] = pd.to_numeric(AMB_PA['Prot C react'], errors='coerce')

Mitjana_protcr = AMB_PA['Prot C react'].mean()
desviacio_estandard_protcr = AMB_PA['Prot C react'].std()

resultat = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultat)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 8.90 ± 5.47


In [384]:
AMB_PA_MECVV.loc[:, 'Prot C react'] = pd.to_numeric(AMB_PA_MECVV['Prot C react'], errors='coerce')

Mitjana_protcr = AMB_PA_MECVV['Prot C react'].mean()
desviacio_estandard_protcr = AMB_PA_MECVV['Prot C react'].std()

resultat = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultat)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 6.38 ± 3.51


In [385]:
SENSE_PA.loc[:, 'Prot C react'] = pd.to_numeric(SENSE_PA['Prot C react'], errors='coerce')

Mitjana_protcr = SENSE_PA['Prot C react'].mean()
desviacio_estandard_protcr = SENSE_PA['Prot C react'].std()

resultat = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultat)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 5.13 ± 5.30


In [386]:
TOTAL.loc[:, 'Prot C react'] = pd.to_numeric(TOTAL['Prot C react'], errors='coerce')

Mitjana_protcr = TOTAL['Prot C react'].mean()
desviacio_estandard_protcr = TOTAL['Prot C react'].std()

resultat = f"{Mitjana_protcr:.2f} ± {desviacio_estandard_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultat)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 6.30 ± 6.09


#### Pvalor

##### Normalitat

In [387]:
pcrambpa=AMB_PA['Prot C react'].dropna()
stat, p_value = shapiro(pcrambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9423
Valor p = 0.60665876
El nombre de files segueix una distribució normal.


In [388]:
pcrambpamecvv=AMB_PA_MECVV['Prot C react'].dropna()
stat, p_value = shapiro(pcrambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8067
Valor p = 0.13058912
El nombre de files segueix una distribució normal.


In [389]:
pcrsensepa=SENSE_PA['Prot C react'].dropna()
stat, p_value = shapiro(pcrsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8945
Valor p = 0.34256857
El nombre de files segueix una distribució normal.


In [390]:
pcrtotal=TOTAL['Prot C react'].dropna()
stat, p_value = shapiro(pcrtotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8808
Valor p = 0.00862808
El nombre de files no segueix una distribució normal.


In [391]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Prot C react'], AMB_PA_MECVV['Prot C react'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [392]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Prot C react'], SENSE_PA['Prot C react'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [393]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Prot C react'], SENSE_PA['Prot C react'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [394]:
t_stat_14, p_val_14 = mannwhitneyu(AMB_PA['Prot C react'], TOTAL['Prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [395]:
t_stat_24, p_val_24 = mannwhitneyu(AMB_PA_MECVV['Prot C react'], TOTAL['Prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [396]:
t_stat_34, p_val_34 = mannwhitneyu(SENSE_PA['Prot C react'], TOTAL['Prot C react'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Urea 

In [397]:
AMB_PA.loc[:, 'Urea'] = pd.to_numeric(AMB_PA['Urea'], errors='coerce')

Mitjana_Urea = AMB_PA['Urea'].mean()
desviacio_estandard_Urea = AMB_PA['Urea'].std()

resultat = f"{Mitjana_Urea:.2f} ± {desviacio_estandard_Urea:.2f}"
print("Mitjana ± Desviació Estàndard de la Urea:", resultat)

Mitjana ± Desviació Estàndard de la Urea: 62.17 ± 36.37


In [398]:
AMB_PA_MECVV.loc[:, 'Urea'] = pd.to_numeric(AMB_PA_MECVV['Urea'], errors='coerce')

Mitjana_Urea = AMB_PA_MECVV['Urea'].mean()
desviacio_estandard_Urea = AMB_PA_MECVV['Urea'].std()

resultat = f"{Mitjana_Urea:.2f} ± {desviacio_estandard_Urea:.2f}"
print("Mitjana ± Desviació Estàndard de la Urea:", resultat)

Mitjana ± Desviació Estàndard de la Urea: 82.27 ± 32.15


In [399]:
SENSE_PA.loc[:, 'Urea'] = pd.to_numeric(SENSE_PA['Urea'], errors='coerce')

Mitjana_Urea = SENSE_PA['Urea'].mean()
desviacio_estandard_Urea = SENSE_PA['Urea'].std()

resultat = f"{Mitjana_Urea:.2f} ± {desviacio_estandard_Urea:.2f}"
print("Mitjana ± Desviació Estàndard de la Urea:", resultat)

Mitjana ± Desviació Estàndard de la Urea: 43.62 ± 18.84


In [400]:
TOTAL.loc[:, 'Urea'] = pd.to_numeric(TOTAL['Urea'], errors='coerce')

Mitjana_Urea = TOTAL['Urea'].mean()
desviacio_estandard_Urea = TOTAL['Urea'].std()

resultat = f"{Mitjana_Urea:.2f} ± {desviacio_estandard_Urea:.2f}"
print("Mitjana ± Desviació Estàndard de la Urea:", resultat)

Mitjana ± Desviació Estàndard de la Urea: 54.02 ± 30.40


#### Pvalor

##### Normalitat

In [401]:
uambpa=AMB_PA['Urea'].dropna()
stat, p_value = shapiro(uambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8842
Valor p = 0.17374314
El nombre de files segueix una distribució normal.


In [402]:
uambpamecvv=AMB_PA_MECVV['Urea'].dropna()
stat, p_value = shapiro(uambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9999
Valor p = 0.97770778
El nombre de files segueix una distribució normal.


In [403]:
usensepa=SENSE_PA['Urea'].dropna()
stat, p_value = shapiro(usensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8322
Valor p = 0.11225495
El nombre de files segueix una distribució normal.


In [404]:
utotal=TOTAL['Urea'].dropna()
stat, p_value = shapiro(utotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8161
Valor p = 0.00054555
El nombre de files no segueix una distribució normal.


In [405]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Urea'], AMB_PA_MECVV['Urea'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [406]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Urea'], SENSE_PA['Urea'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [407]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Urea'], SENSE_PA['Urea'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [408]:
t_stat_14, p_val_14 = mannwhitneyu(AMB_PA['Urea'], TOTAL['Urea'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [409]:
t_stat_24, p_val_24 = mannwhitneyu(AMB_PA_MECVV['Urea'], TOTAL['Urea'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [410]:
t_stat_34, p_val_34 = mannwhitneyu(SENSE_PA['Urea'], TOTAL['Urea'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Filtrat glomerular CDK-EPI

In [411]:
AMB_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = AMB_PA['FGE CDK-EPI'].mean()
desviacio_estandard_fg = AMB_PA['FGE CDK-EPI'].std()

resultat = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultat)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 56.00 ± 12.65


In [412]:
AMB_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA_MECVV['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = AMB_PA_MECVV['FGE CDK-EPI'].mean()
desviacio_estandard_fg = AMB_PA_MECVV['FGE CDK-EPI'].std()

resultat = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultat)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 48.00 ± 2.00


In [413]:
SENSE_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SENSE_PA['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = SENSE_PA['FGE CDK-EPI'].mean()
desviacio_estandard_fg = SENSE_PA['FGE CDK-EPI'].std()

resultat = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultat)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 57.60 ± 16.27


In [414]:
TOTAL.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(TOTAL['FGE CDK-EPI'], errors='coerce')

Mitjana_fg = TOTAL['FGE CDK-EPI'].mean()
desviacio_estandard_fg = TOTAL['FGE CDK-EPI'].std()

resultat = f"{Mitjana_fg:.2f} ± {desviacio_estandard_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultat)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 59.94 ± 18.70


#### Pvalor

##### Normalitat

In [415]:
fgepiambpa=AMB_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7690
Valor p = 0.01994941
El nombre de files no segueix una distribució normal.


In [416]:
fgepiambpamecvv=AMB_PA_MECVV['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


In [417]:
fgepisensepa=SENSE_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepisensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9605
Valor p = 0.81120317
El nombre de files segueix una distribució normal.


In [418]:
fgepitotal=TOTAL['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepitotal)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9054
Valor p = 0.08369069
El nombre de files segueix una distribució normal.


In [419]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], AMB_PA_MECVV['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [420]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [421]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


In [422]:
t_stat_14, p_val_14 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], TOTAL['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL:", p_val_14)

P-valor entre AMB_PA i TOTAL: nan


In [423]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['FGE CDK-EPI'], TOTAL['FGE CDK-EPI'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAL: nan


In [424]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['FGE CDK-EPI'], TOTAL['FGE CDK-EPI'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan


### Filtrat glomerular MDRD

In [425]:
AMB_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA['FGE MDRD'], errors='coerce')

Mitjana_fgmd = AMB_PA['FGE MDRD'].mean()
desviacio_estandard_fgmd = AMB_PA['FGE MDRD'].std()

resultat = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultat)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 51.50 ± 0.71


In [426]:
AMB_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA_MECVV['FGE MDRD'], errors='coerce')

Mitjana_fgmd = AMB_PA_MECVV['FGE MDRD'].mean()
desviacio_estandard_fgmd = AMB_PA_MECVV['FGE MDRD'].std()

resultat = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultat)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 51.00 ± nan


In [427]:
SENSE_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SENSE_PA['FGE MDRD'], errors='coerce')

Mitjana_fgmd = SENSE_PA['FGE MDRD'].mean()
desviacio_estandard_fgmd = SENSE_PA['FGE MDRD'].std()

resultat = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultat)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 39.00 ± nan


In [428]:
TOTAL.loc[:, 'FGE MDRD'] = pd.to_numeric(TOTAL['FGE MDRD'], errors='coerce')

Mitjana_fgmd = TOTAL['FGE MDRD'].mean()
desviacio_estandard_fgmd = TOTAL['FGE MDRD'].std()

resultat = f"{Mitjana_fgmd:.2f} ± {desviacio_estandard_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultat)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 37.67 ± 17.43


#### Pvalor

##### Normalitat

In [429]:
stat, p_value = shapiro(AMB_PA['FGE MDRD'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")


Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [430]:
stat, p_value = shapiro(AMB_PA_MECVV['FGE MDRD'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [431]:
stat, p_value = shapiro(SENSE_PA['FGE MDRD'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [432]:
stat, p_value = shapiro(TOTAL['FGE MDRD'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [433]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['FGE MDRD'], AMB_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [434]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [435]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i AMB_PA_MECVV:", p_val_23)

P-valor entre AMB_PA_MECVV i AMB_PA_MECVV: nan


In [436]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA['FGE MDRD'], TOTAL['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i TOTAl:", p_val_14)

P-valor entre AMB_PA i TOTAl: nan


In [437]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV['FGE MDRD'], TOTAL['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i TOTAl:", p_val_24)

P-valor entre AMB_PA_MECVV i TOTAl: nan


In [438]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA['FGE MDRD'], TOTAL['FGE MDRD'], equal_var=False)
print("P-valor entre SENSE_PA i TOTAL:", p_val_34)

P-valor entre SENSE_PA i TOTAL: nan
